In [5]:
%pip install azureml-opendatasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.4/455.4 MB 34.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... - \ done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.0 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 96.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-4.1.1-py2.py3-none-any.whl size=456008706 sha256=d5b48e6b7be524ca9de894601c537d29124ab753026d19d4af11ceeabdb1a957
  Stored in directory: /home/azureuser/.cache/pip/wheels/16/77/d3/d15aaaab1df8384ad9bd94caba26a1a5aa439d8afd187a5ab9
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10540 sha256=1f866366330f26f044844b1fd973277fe340a28166aa41ee9707aaa95b0598a8
  Stored in directory: /home/azureuser/.cache/pip/wheels/c0/18/f6/f0d6be9d0435e2677ce5cc758e91da50053dce456a346f08c5
Successfully built pyspark fusepy
  Attempting uninstall: azureml-core
    Found existing installation: azu

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Authenticate and connect to the workspace
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

print(f"Connected to workspace: {ml_client.workspace_name}")

Found the config file in: /config.json


Connected to workspace: rk-ml-2026-workspace


In [6]:
import pandas as pd
from azureml.opendatasets import Diabetes

# Fetch the dataset
diabetes_data = Diabetes.get_tabular_dataset()
df = diabetes_data.to_pandas_dataframe()

# Show the first few rows
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,135


In [8]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression  # Changed to Linear
from sklearn.metrics import mean_squared_error, r2_score

mlflow.set_experiment("diabetes-regression") # New experiment name

with mlflow.start_run():
    X = df.drop('Y', axis=1)
    y = df['Y']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 1. Train Linear Regression Model
    model = LinearRegression().fit(X_train, y_train)

    # 2. Evaluate
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    print(f"Mean Squared Error: {mse}")
    print(f"R2 Score: {r2}")

    # 3. Log with MLflow
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)
    mlflow.sklearn.log_model(model, "model")

from mlflow.models.signature import infer_signature

# ... after training the model ...
signature = infer_signature(X_test, predictions)

# Update your log_model line:
mlflow.sklearn.log_model(model, "model", signature=signature)

Mean Squared Error: 2900.193628493482
R2 Score: 0.45260276297191937
🏃 View run dreamy_brake_hf8ntrph at: https://eastasia.api.azureml.ms/mlflow/v2.0/subscriptions/609834a1-addc-4de7-a3f0-1b00a4e8a701/resourceGroups/rk-ml-2026/providers/Microsoft.MachineLearningServices/workspaces/rk-ml-2026-workspace/#/experiments/e34f06b1-79f8-44b0-a8b6-6f2ecc1ac41d/runs/02d6a6c1-ec65-4f9e-81a0-957d544b1c30
🧪 View experiment at: https://eastasia.api.azureml.ms/mlflow/v2.0/subscriptions/609834a1-addc-4de7-a3f0-1b00a4e8a701/resourceGroups/rk-ml-2026/providers/Microsoft.MachineLearningServices/workspaces/rk-ml-2026-workspace/#/experiments/e34f06b1-79f8-44b0-a8b6-6f2ecc1ac41d


2026/01/10 09:54:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
